# Install and Import Libs

In [1]:
!pip install --upgrade gensim==4.3.2 smart_open==6.4.0
!pip install langchain langchain-community chromadb sentence-transformers gensim nltk tqdm --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 35.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: smart_open
    Found existing installation: smart-open 7.1.0
    Uninstalling smart-open-7.1.0:
      Successfully uninstalled smart-open-7.1.0
  Attempting uninstall: gensim
    Found existing installation: gensim 4.3.3
    Uninstalling gensim-4.3.3:
      Successfully uninstalled gensim-4.3.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.4 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 61.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip uninstall -y protobuf google-cloud-storage google-cloud-core google-cloud-automl
!pip install protobuf==3.20.3

Found existing installation: protobuf 6.32.1
Uninstalling protobuf-6.32.1:
  Successfully uninstalled protobuf-6.32.1
Found existing installation: google-cloud-storage 2.19.0
Uninstalling google-cloud-storage-2.19.0:
  Successfully uninstalled google-cloud-storage-2.19.0
Found existing installation: google-cloud-core 2.4.3
Uninstalling google-cloud-core-2.4.3:
  Successfully uninstalled google-cloud-core-2.4.3
Found existing installation: google-cloud-automl 1.0.1
Uninstalling google-cloud-automl-1.0.1:
  Successfully uninstalled google-cloud-automl-1.0.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 3.2 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-translate 3.12.1 requires google-cloud-core<3.0.0dev,>=1.4.4, which is not installed.
google-cloud-datastore 2.21.0 requires google-cloud-core<3.0.0,>=1.4.0, whi

In [3]:
!pip install --force-reinstall "scipy==1.10.1" 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 1.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 45.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 64.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-aiplatform 1.99.0 requires google-cloud-storage<3.0.0,>=1.32.0, which is not installed.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, wh

In [4]:
import os, re, json, pickle
from tqdm import tqdm
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Utility Function

In [5]:
STOPWORDS = set(stopwords.words('english'))

def read_json_file(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', ' ', text)
    tokens = [w for w in word_tokenize(text) if w not in STOPWORDS and len(w) > 2]
    return " ".join(tokens)


# Preprocess and NMF Model

In [6]:
import gensim
import gensim.corpora as corpora
from gensim.models.phrases import Phrases, Phraser
from gensim.models.nmf import Nmf


def preprocess_texts(texts):
    lemmatizer = WordNetLemmatizer()
    tokenized = [text.split() for text in texts]
    bigram = Phrases(tokenized, min_count=5, threshold=100)
    bigram_mod = Phraser(bigram)
    tokenized = [bigram_mod[doc] for doc in tokenized]
    data_lemmatized = [[lemmatizer.lemmatize(w) for w in doc] for doc in tokenized]
    return data_lemmatized

def build_nmf_model(data_lemmatized, num_topics=20):
    id2word = corpora.Dictionary(data_lemmatized)
    corpus = [id2word.doc2bow(text) for text in data_lemmatized]
    nmf_model = Nmf(corpus=corpus, num_topics=num_topics, id2word=id2word, random_state=42)
    return nmf_model, id2word, corpus


# Load Metadata

In [8]:
data_path = "/kaggle/input/hepth-metadata-file/abs_metadata.json"
papers = read_json_file(data_path)

texts = [clean_text(p["abstract"]) for p in papers]
print(f"Loaded {len(texts)} abstracts.")


Loaded 29555 abstracts.


# NMF TOPIC MODEL

In [9]:
import os

os.makedirs("data", exist_ok=True)


In [10]:
data_lemmatized = preprocess_texts(texts)
nmf_model, id2word, corpus = build_nmf_model(data_lemmatized, num_topics=20)

# Lưu model
os.makedirs("data", exist_ok=True)
with open("data/nmf_model.pkl", "wb") as f:
    pickle.dump(nmf_model, f)
with open("data/id2word.pkl", "wb") as f:
    pickle.dump(id2word, f)

print("NMF model built successfully!")


NMF model built successfully!


# CREAT EMBEDDING AND CHROMADB

In [11]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Tạo 2 collection: abstract và topic
collection = Chroma(
    persist_directory="data/chroma_collection",
    embedding_function=embed_model
)
topic_collection = Chroma(
    persist_directory="data/chroma_topic_collection",
    embedding_function=embed_model
)

print("ChromaDB collections ready.")


/tmp/ipykernel_36/3202640238.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
2025-10-04 16:51:25.776859: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759596686.107944      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759596686.194482      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipykernel_36/3202640238.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  collection = Chroma(


ChromaDB collections ready.



# RETRIEVER SEMANTIC & TOPIC


In [12]:

from langchain_text_splitters import RecursiveCharacterTextSplitter

class TopicChunkedChromaRetriever:
    def __init__(self, collection, topic_collection, nmf_model, id2word, embed_model,
                 top_k_docs=5, chunk_size=500, chunk_overlap=50, top_k_chunks=5):
        self.collection = collection
        self.topic_collection = topic_collection
        self.nmf_model = nmf_model
        self.id2word = id2word
        self.embed_model = embed_model
        self.top_k_docs = top_k_docs
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.top_k_chunks = top_k_chunks
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=self.chunk_size,
            chunk_overlap=self.chunk_overlap
        )

    def get_topic_vector(self, text):
        tokens = text.lower().split()
        bow = self.id2word.doc2bow(tokens)
        num_topics = self.nmf_model.num_topics
        topics_sparse = dict(self.nmf_model.get_document_topics(bow, minimum_probability=0.0))
        dense_vector = [topics_sparse.get(i, 0.0) for i in range(num_topics)]
        return dense_vector

    def retrieve(self, query):
        dense_vector = self.get_topic_vector(query)
        results = self.topic_collection.query(
            query_embeddings=[dense_vector],
            n_results=50,
            include=["documents", "ids"]
        )
        ids = results["ids"][0]

        query_embedding = self.embed_model.embed_query(query)
        results = self.collection.query(
            query_embeddings=[query_embedding],
            n_results=self.top_k_docs,
            where={"paper_id": {"$in": ids}},
            include=["documents"]
        )

        return results["documents"][0]


# LLM CHAIN + PROMPT

In [13]:
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import LLMChain

def make_rag_prompt():
    template = """
    You are a helpful academic assistant. Based on the retrieved documents below, answer the user's question.

    === Retrieved Documents ===
    {context}

    === Question ===
    {question}

    Provide a concise and factual answer based on the context.
    """
    return ChatPromptTemplate.from_template(template)

def build_rag_chain():
    prompt = make_rag_prompt()
    llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.3)
    chain = LLMChain(prompt=prompt, llm=llm)
    return chain


# Pipeline

In [ ]:
retriever = TopicChunkedChromaRetriever(
    collection=collection,
    topic_collection=topic_collection,
    nmf_model=nmf_model,
    id2word=id2word,
    embed_model=embed_model
)

rag_chain = build_rag_chain()

query = "What are recent advances in diffusion models for image generation?"
docs = retriever.retrieve(query)
context = "\n".join(docs)

answer = rag_chain.run({"context": context, "question": query})
print("=== ANSWER ===\n", answer)
